In [ ]:
from xas import find_similar_measurements

inpath = "/dls/i06-1/data/2025/cm40613-2//i06-1-353230.nxs"
outpath = "output.nxs"

# XMCD Notebook
Load a sequence of scans using a dummy scan to normalise, correct and subtract polarised x-ray absorption spectra.

```python
scan dummy -5 -1 1 xmcd_processor  # assumes previous 4 scans were energy scans at different polarisations
```

### Note
**mmg_toolbox** is in active development and likely to change in the future, therefore please don't rely to heavily on the functions at this point. You can see how everything works at the [repository](https://github.com/DiamondLightSource/mmg_toolbox). If you have any comments, please do get in touch!

In [ ]:
import os
import matplotlib.pyplot as plt

from mmg_toolbox import xas, data_file_reader, module_info
from mmg_toolbox.utils.file_functions import replace_scan_number
from mmg_toolbox.xas.nxxas_loader import is_nxxas
from mmg_toolbox.xas.spectra_container import average_polarised_scans

print(module_info())


In [ ]:
dummy_scan = data_file_reader(inpath)

scan_numbers = dummy_scan('axes')
cmd = dummy_scan('(cmd|user_command|scan_command)')
scan_number = dummy_scan.scan_number()

print('Dummy scan:')
print(f"{dummy_scan.filename}  scan_number = {scan_number}, cmd = '{cmd}'\n")

# handle relative scan numbers (0 is current scan)
scan_numbers = [n if n > 100 else scan_number + n for n in scan_numbers]
scan_files = [replace_scan_number(inpath, int(n)) for n in scan_numbers]
scan_files = [f for f in scan_files if is_nxxas(f)]  # only measure

print('Scan files:')
print('\n'.join(scan_files))


In [ ]:
# Load scan data and plot raw spectra
scans = xas.load_xas_scans(*scan_files)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[12, 6], dpi=100)
for scan in scans:
    print(scan, '\n')
    tey, tfy = scan.spectra.values()
    tey.plot(ax1)
    tfy.plot(ax2)

ax1.set_xlabel('E [eV]')
ax1.set_ylabel('TEY / monitor')
ax2.set_xlabel('E [eV]')
ax2.set_ylabel('TFY / monitor')
ax1.legend()
ax2.legend()

In [ ]:
# check previous ~10 scans and build a list of energy scans at the same edge, temperature and field
# see help(find_similar_measurements) for more info
pols = {s.metadata.pol for s in scans}

print(f"Unique polarisations: {pols}")

if len(pols) > 1:
    for xas_scan in scans:
        m = xas_scan.metadata
        print(f"{m.scan_no} T={m.temp:3.0f}K, B={m.mag_field:4.1f}T,  {m.pol}")
else:
    raise Exception('Not enough polarisations')

In [ ]:
for xas_scan in scans:
    xas_scan.divide_by_preedge()
    print(xas_scan)

# plot scan normalised scan files
fig, axes = plt.subplots(1, 2, figsize=(12, 4), dpi=100)
fig.suptitle('Normalise by pre-edge')
for xas_scan in scans:
    for n, (mode, spectra) in enumerate(xas_scan.spectra.items()):
        spectra.plot(ax=axes[n], label=f"{xas_scan.name} {xas_scan.metadata.pol}")
        axes[n].set_ylabel(mode)

for ax in axes.flat:
    ax.set_xlabel('E [eV]')
    ax.legend()

In [ ]:
# Average polarised scans
for xas_scan in scans:
    print(f"{xas_scan.name}: {xas_scan.metadata.pol}")
pol1, pol2 = average_polarised_scans(*scans)
print(pol1)
print(pol2)

# Plot averaged scans
fig, axes = plt.subplots(1, 2, figsize=(12, 4), dpi=100)
fig.suptitle('Averaged polarised scans')
for xas_scan in [pol1, pol2]:
    for n, (mode, spectra) in enumerate(xas_scan.spectra.items()):
        spectra.plot(ax=axes[n], label=xas_scan.name)
        axes[n].set_ylabel(mode)

for ax in axes.flat:
    ax.set_xlabel('E [eV]')
    ax.legend()

# Calculate XMCD/XMLD

In [ ]:
# Calculate XMCD
xmcd = pol1 - pol2
print(xmcd)

for name, spectra in xmcd.spectra.items():
    print(spectra)
    print(spectra.process)
    print(spectra.sum_rules_report(1))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4), dpi=100)
fig.suptitle(xmcd.name.upper())
for n, (mode, spectra) in enumerate(xmcd.spectra.items()):
    spectra.plot(ax=axes[n])
    axes[n].set_ylabel(mode)

for ax in axes.flat:
    ax.set_xlabel('E [eV]')
    ax.legend()

In [ ]:
# Save xmcd file
xmcd_filename = f"{scans[0].metadata.scan_no}-{scans[-1].metadata.scan_no}_{xmcd.name}.nxs"
output_folder = os.path.dirname(outpath)
xmcd.write_nexus(os.path.join(output_folder, xmcd_filename))